In [84]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [85]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [86]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [87]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [88]:
from collections import defaultdict

In [89]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [90]:
len(hashes), len(documents)

(947, 948)

In [91]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [92]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [93]:
import json

In [94]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [95]:
!head documents-with-ids.json

'head' is not recognized as an internal or external command,
operable program or batch file.


In [96]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [97]:
from google import genai

In [98]:
from dotenv import load_dotenv
import os

In [99]:
load_dotenv() 
API_KEY = os.getenv("GEMINI_API_KEY")


In [100]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    client = genai.Client(api_key=API_KEY)
    response = client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt)

    json_response = response.text
    return json_response

In [101]:
# from openai import OpenAI
# client = OpenAI()

In [102]:
# def generate_questions(doc):
#     prompt = prompt_template.format(**doc)

#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )

#     json_response = response.choices[0].message.content
#     return json_response

In [103]:
from tqdm.auto import tqdm

In [104]:
results = {}

In [105]:
results

{}

In [ ]:
# import time
# from tqdm import tqdm

# for doc in tqdm(documents): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue

#     try:
#         questions = generate_questions(doc)
#         results[doc_id] = questions
#         time.sleep(4.1)  # To stay within 15 req/min
#     except Exception as e:
#         print(f"Error for doc_id {doc_id}: {e}")
#         time.sleep(10)  # Extra backoff on error

100%|██████████| 948/948 [1:49:03<00:00,  6.90s/it]


In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

In [80]:
import pickle

In [116]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'results.bin'

In [82]:
results['1f6520ca']

'```json\n[\n  "What specific background knowledge or skills are expected before enrolling in this course?",\n  "Where can I find a detailed list of prerequisites?",\n  "Are there any recommended resources to review before the course begins?",\n  "Do I need prior experience with data engineering concepts to succeed?",\n  "Is there a recommended level of proficiency with tools like Git or GitHub before starting?"\n]\n```'

In [106]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [107]:
doc_index = {d['id']: d for d in documents}

In [109]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [110]:
import pandas as pd

In [111]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [112]:
df.to_csv('ground-truth-data.csv', index=False)

In [113]:
!head ground-truth-data.csv

'head' is not recognized as an internal or external command,
operable program or batch file.
